<div style='float: right'><img src='pic/pipelink.png'/></div>
## <div id='pipelink' />パイプリンク

In [ ]:
import numpy as np
from itertools import product
from pulp import *
from unionfind import unionfind
from ortoolpy import addvar, addvars, addbinvar, addbinvars
data = """\
 . - . . 
. . | . .
 . - . . 
. | | . |
 . . - . 
. . . . |
 - - . . 
. . . . .
 . . . . """.split('\n')
nw, nh = (len(data[0])+1)//2, (len(data)+1)//2

### 問題
* すべてのマスに線をひき１つのリンクにします
* 線は交差してもよいが、枝分かれはしません
* 最初に線が引いてあるマスは形を変えてはいけません

### 変数
* vh (1)
* vv (2)
* vr：表示用 (3)

### 制約
* 指定した形はそのままとすること (4)
* 全マスに線を引くこと (5)
* 交差はいいが、枝分かれしないこと (6)
* １つのリンクにすること (7)
* 外にはみ出ないこと (8)
* vrをvhとvvで表現 (9)

In [ ]:
m = LpProblem()
vh = np.array(addbinvars(nh, nw+1)) # (1)
vv = np.array(addbinvars(nh+1, nw)) # (2)
vr = np.array(addvars(nh, nw)) # (3)
vh[:,0] = vh[:,nw] = vv[0,:] = vv[nh,:] = 0 # (8)
for i, j in product(range(nh), range(nw)):
    if j > 0 and data[2*i][2*j-1] == '-':
        m += vh[i,j] == 1 # (4)
    if i > 0 and data[2*i-1][2*j] == '|':
        m += vv[i,j] == 1 # (4)
    l = [vv[i,j], vh[i,j], vv[i+1,j], vh[i,j+1]]
    m += lpSum(l) >= 2 # (5)
    for x in l:
        m += lpSum(l) <= 2+2*x # (6)
    m += lpDot(np.power(2, np.arange(4)), l) == vr[i,j] # (9)
while True:
    %time m.solve()
    rh = np.vectorize(value)(vh).astype(int)
    rv = np.vectorize(value)(vv).astype(int)
    rr = np.vectorize(value)(vr).astype(int)
    b = rr == 15
    u = unionfind(nh * nw)
    for i, j in product(range(nh), range(nw)):
        if b[i,j]:
            u.unite(i - 1 + j * nw, i + 1 + j * nw)
            u.unite(i + j * nw - nw, i + j * nw + nw)
        else:
            if rv[i,j] and (i == 0 or not b[i-1,j]):
                u.unite(i + j * nw, i - 1 + j * nw)
            if rh[i,j] and (j == 0 or not b[i,j-1]):
                u.unite(i + j * nw, i + j * nw - nw)
    if all([b[i,j] or u.issame(0, i + j * nw) for i in range(nh) for j in range(nw)]):
        break
    for gr in u.groups():
        if len(gr) == 1:
            continue
        l = []
        for k in gr:
            i, j = k % nw, k // nw
            l.extend(vv[i+p,j] for p in range(2) if rv[i+p,j])
            l.extend(vh[i,j+p] for p in range(2) if rh[i,j+p])
        m += lpSum(l) <= len(l) - 2 # (7)
print('\n'.join(''.join(s) for s in np.vectorize(lambda x: '012┘4│┐78└─1┌34┼'[x])(rr)))